In [2]:
import pandas as pd

# Function to extend the normalization of team names
def extended_normalize_team_name(name):
    # Extended replacements
    extended_replacements = {
        "Alaves": "Alavés",
        "Man City": "Manchester City",
        "Man United": "Manchester Utd",
        "Nott'm Forest": "Nott'ham Forest",
        "Ath Bilbao": "Athletic Club",
        "Ath Madrid": "Atlético Madrid",
        "Cadiz": "Cádiz",
        "Celta": "Celta Vigo",
        "Clermont": "Clermont Foot",
        "Darmstadt": "Darmstadt 98",
        "Ein Frankfurt": "Eint Frankfurt",
        "FC Koln": "Köln",
        "Famalicao": "Famalicão",
        "For Sittard": "Fortuna Sittard",
        "Gil Vicente": "Gil Vicente FC",
        "Go Ahead Eagles": "Go Ahead Eag",
        "Guimaraes": "Vitória",
        "Heracles": "Heracles Almelo",
        "Luton": "Luton Town",
        "M'gladbach": "M'Gladbach",
        "Mainz": "Mainz 05",
        "Newcastle": "Newcastle Utd",
        "Nijmegen": "NEC Nijmegen",
        "Oud-Heverlee Leuven": "OH Leuven",
        "Paris SG": "Paris S-G",
        "Sheffield United": "Sheffield Utd",
        "Sociedad": "Real Sociedad",
        "Sp Braga": "Braga",
        "Sp Lisbon": "Sporting CP",
        "Sparta Rotterdam": "Sparta R'dam",
        "St Truiden": "Sint-Truiden",
        "St. Gilloise": "Union SG",
        "Standard": "Standard Liège",
        "Vallecano": "Rayo Vallecano",
        "Verona": "Hellas Verona",
        "Waalwijk": "RKC Waalwijk",
        "Luton":"Luton Town"
    }
    return extended_replacements.get(name, name)

# Function for final normalization adjustments
def final_normalize_team_name(name):
    final_replacements = {
        "Almeria": "Almería",
        "Nottingham Forest": "Nott'ham Forest",
        "Luton":"Luton Town"
    }
    return final_replacements.get(name, name)

# Loading the 'all_euro_master_data' and 'combined_football_league_stats' dataframes
all_euro_path = "all_euro_master_data.csv"
combined_league_path = "combined_football_league_stats.csv"
all_euro_df = pd.read_csv(all_euro_path)
combined_league_df = pd.read_csv(combined_league_path)

# Applying the normalization to team names
all_euro_df['HomeTeam'] = all_euro_df['HomeTeam'].apply(extended_normalize_team_name)
all_euro_df['AwayTeam'] = all_euro_df['AwayTeam'].apply(extended_normalize_team_name)
combined_league_df['Squad'] = combined_league_df['Squad'].apply(extended_normalize_team_name)

# Final adjustments for specific mismatches
all_euro_df['HomeTeam'] = all_euro_df['HomeTeam'].apply(final_normalize_team_name)
all_euro_df['AwayTeam'] = all_euro_df['AwayTeam'].apply(final_normalize_team_name)
combined_league_df['Squad'] = combined_league_df['Squad'].apply(final_normalize_team_name)

# Merging the dataframes
merged_data = pd.merge(all_euro_df, combined_league_df, left_on="HomeTeam", right_on="Squad", how="left", suffixes=('', '_Home'))
merged_data = pd.merge(merged_data, combined_league_df, left_on="AwayTeam", right_on="Squad", how="left", suffixes=('_Home', '_Away'))

# Saving the merged dataframe to a CSV file
final_merged_output_file_path = 'final_merged_all_euro_combined_league.csv'
merged_data.to_csv(final_merged_output_file_path, index=False)


In [9]:
import pandas as pd
from difflib import get_close_matches

# Load the provided files
file_euro = "final_merged_all_euro_combined_league.csv"
file_league = "modified_league_ranking.csv"

# Read the files into dataframes
df_euro = pd.read_csv(file_euro)
df_league = pd.read_csv(file_league)

# Extract unique team names from both datasets
unique_teams_euro = set(map(str, df_euro['HomeTeam'].unique())).union(set(df_euro['AwayTeam'].unique()))
unique_teams_league = set(map(str, df_league['Club'].unique()))

# Fuzzy string matching for team names
team_name_mapping = {team: get_close_matches(team, unique_teams_league, n=1, cutoff=0.6) for team in unique_teams_euro}

# Apply the initial mappings to the Euro dataset
df_euro['HomeTeam'] = df_euro['HomeTeam'].apply(lambda team: next(iter(team_name_mapping.get(str(team), [])), team))
df_euro['AwayTeam'] = df_euro['AwayTeam'].apply(lambda team: next(iter(team_name_mapping.get(str(team), [])), team))

# Manual mapping for unmatched team names
manual_mapping = {
    'Ajax': 'Ajax Amsterdam',
    'Alavés': 'Deportivo Alaves',
    'Brighton': 'Brighton & Hove Albion',
    'Köln': '1. FC Köln',
    "M'Gladbach": 'Borussia Mönchengladbach',
    'Hearts': 'Heart of Midlothian',
    'Luton':'Luton Town'
}

# Update team names in the Euro dataframe
df_euro['HomeTeam'] = df_euro['HomeTeam'].replace(manual_mapping)
df_euro['AwayTeam'] = df_euro['AwayTeam'].replace(manual_mapping)

# Merge the ranking data into the all Euro data
df_euro_home = df_euro.copy()
df_euro_away = df_euro.copy()

df_euro_home['Club'] = df_euro_home['HomeTeam']
df_euro_away['Club'] = df_euro_away['AwayTeam']

df_euro_home_merged = df_euro_home.merge(df_league, on='Club', how='left', suffixes=('', '_HomeRanking'))
df_euro_away_merged = df_euro_away.merge(df_league, on='Club', how='left', suffixes=('', '_AwayRanking'))

df_euro_away_merged = df_euro_away_merged.drop(['Club'], axis=1)
df_final_merged = df_euro_home_merged.merge(df_euro_away_merged, on=list(df_euro.columns), how='left')

# Rename the columns for rankings and points
df_final_merged.rename(columns={
    'Rank_x': 'RankHome',
    'Points_x': 'PointsHome',
    'Rank_y': 'RankAway',
    'Points_y': 'PointsAway'
}, inplace=True)

# Save the final merged dataframe to a new CSV file
final_renamed_file_path = "final_all_euro_data_with_rankings.csv"
df_final_merged.to_csv(final_renamed_file_path, index=False)


In [10]:
# Creating a mapping of the current column names to their full abbreviations as specified in the text files
full_abbreviation_mapping = {
    # Match and Team Information
    'Div': 'Division', 'Date': 'Date', 'Time': 'Time', 'HomeTeam': 'Home Team', 'AwayTeam': 'Away Team',

    # Match Results and Statistics
    'FTHG': 'Full Time Home Goals', 'FTAG': 'Full Time Away Goals', 'FTR': 'Full Time Result',
    'HTHG': 'Half Time Home Goals', 'HTAG': 'Half Time Away Goals', 'HTR': 'Half Time Result',
    'Referee': 'Referee', 'HS': 'Home Shots', 'AS': 'Away Shots', 'HST': 'Home Shots on Target',
    'AST': 'Away Shots on Target', 'HF': 'Home Fouls', 'AF': 'Away Fouls', 'HC': 'Home Corners',
    'AC': 'Away Corners', 'HY': 'Home Yellow Cards', 'AY': 'Away Yellow Cards', 'HR': 'Home Red Cards',
    'AR': 'Away Red Cards',

    # Betting Odds
    'B365H': 'Bet365 Home Win Odds', 'B365D': 'Bet365 Draw Odds', 'B365A': 'Bet365 Away Win Odds',
    'BWH': 'Bet&Win Home Win Odds', 'BWD': 'Bet&Win Draw Odds', 'BWA': 'Bet&Win Away Win Odds',
    'IWH': 'Interwetten Home Win Odds', 'IWD': 'Interwetten Draw Odds', 'IWA': 'Interwetten Away Win Odds',
    'PSH': 'Pinnacle Home Win Odds', 'PSD': 'Pinnacle Draw Odds', 'PSA': 'Pinnacle Away Win Odds',
    'WHH': 'William Hill Home Win Odds', 'WHD': 'William Hill Draw Odds', 'WHA': 'William Hill Away Win Odds',
    'VCH': 'VC Bet Home Win Odds', 'VCD': 'VC Bet Draw Odds', 'VCA': 'VC Bet Away Win Odds',
    'MaxH': 'Market Maximum Home Win Odds', 'MaxD': 'Market Maximum Draw Odds', 'MaxA': 'Market Maximum Away Win Odds',
    'AvgH': 'Market Average Home Win Odds', 'AvgD': 'Market Average Draw Odds', 'AvgA': 'Market Average Away Win Odds',

    # Additional Columns
    'Club': 'Club', 'RankHome': 'Rank Home', 'PointsHome': 'Points Home', 'RankAway': 'Rank Away', 'PointsAway': 'Points Away'
}

# Identifying columns to be renamed based on '_home' and '_away' suffixes
home_columns = [col for col in df_final_merged.columns if '_Home' in col]
away_columns = [col for col in df_final_merged.columns if '_Away' in col]

# Renaming these columns by removing '_Home' and '_Away' suffixes
# and appending '(Home)' and '(Away)' respectively
home_renamed = {col: col.replace('_Home', ' (Home)') for col in home_columns}
away_renamed = {col: col.replace('_Away', ' (Away)') for col in away_columns}

# Applying the renaming
df_final = df_final_merged.rename(columns={**home_renamed, **away_renamed})

# Verifying the changes
final_column_names = df_final.columns.tolist()
final_column_names

# Applying the renaming to the dataframe
df_full_abbreviated = df_final.rename(columns=full_abbreviation_mapping)

# Verifying the changes
full_abbreviated_column_names = df_full_abbreviated.columns.tolist()
full_abbreviated_column_names = 'final_all_euro_data_with_full_abbreviations.csv'
df_full_abbreviated.to_csv(full_abbreviated_column_names, index=False)

In [11]:
# Loading the CSV file
csv_file_path = 'final_all_euro_data_with_full_abbreviations.csv'
df = pd.read_csv(csv_file_path)

# Home and Away Attack Strength
home_attack_strength = df.groupby('Home Team')['Full Time Home Goals'].mean().rename('Home Attack Strength')
away_attack_strength = df.groupby('Away Team')['Full Time Away Goals'].mean().rename('Away Attack Strength')
df = df.merge(home_attack_strength, how='left', left_on='Home Team', right_index=True)
df = df.merge(away_attack_strength, how='left', left_on='Away Team', right_index=True)



# Total shots (home + away)
df['Total Shots'] = df['Home Shots'] + df['Away Shots']

# Total corners (home + away)
df['Total Corners'] = df['Home Corners'] + df['Away Corners']

# Attack strength difference (home - away)
df['Attack Strength Difference'] = df['Home Attack Strength'] - df['Away Attack Strength']

# Goal difference (home and away)
df['Goal Difference Home'] = df['GF (Home)'] - df['GA (Home)']
df['Goal Difference Away'] = df['GF (Away)'] - df['GA (Away)']



# Home and Away Odds
home_odds_columns = [col for col in df.columns if 'Home Win Odds' in col]
away_odds_columns = [col for col in df.columns if 'Away Win Odds' in col]
df['Average Home Odds'] = df[home_odds_columns].mean(axis=1)
df['Average Away Odds'] = df[away_odds_columns].mean(axis=1)



# Function to calculate win, loss, and draw percentages
def calculate_result_percentages(last_5_results):
    if not isinstance(last_5_results, str):
        return np.nan, np.nan, np.nan
    results = last_5_results.split()
    win_percentage = results.count('W') / len(results) * 100
    loss_percentage = results.count('L') / len(results) * 100
    draw_percentage = results.count('D') / len(results) * 100
    return win_percentage, loss_percentage, draw_percentage



# Applying the function to calculate percentages
df['Home Win %'], df['Home Loss %'], df['Home Draw %'] = zip(*df['Last 5 (Home)'].apply(calculate_result_percentages))
df['Away Win %'], df['Away Loss %'], df['Away Draw %'] = zip(*df['Last 5 (Away)'].apply(calculate_result_percentages))

# Displaying a sample of the dataframe
df[['Home Team', 'Away Team', 'Last 5 (Home)', 'Home Win %', 'Home Loss %', 'Home Draw %', 'Last 5 (Away)', 'Away Win %', 'Away Loss %', 'Away Draw %']].head()

# 1. Difference in Team Rankings
df['Ranking_Difference'] = abs(df['Rank Home'] - df['Rank Away'])

# 2. Interaction between Home Team Defense Strength and Away Team Attack Strength
# Assuming 'Home Attack Strength' and 'Away Attack Strength' are indicators of attack strength.
# Defense strength is not directly available, so let's create a proxy for it.
# A simple proxy could be 1 / Attack Strength (assuming higher the attack strength, lower the defense strength).
df['Home_Defense_Strength'] = 1 / df['Home Attack Strength']
df['Away_Defense_Strength'] = 1 / df['Away Attack Strength']
df['Defense_Attack_Interaction'] = df['Home_Defense_Strength'] * df['Away Attack Strength']


# Calculating the probability of each match outcome (win, draw, loss) based on the given percentages

# Probability of Home Win
df['Prob Home Win'] = (df['Home Win %'] / 100)

# Probability of Away Win
df['Prob Away Win'] = (df['Away Win %'] / 100)

# Probability of Draw
# Assuming the draw probability can be estimated as 1 - (Prob Home Win + Prob Away Win)
df['Prob Draw'] = 1 - (df['Prob Home Win'] + df['Prob Away Win'])
df['Prob Draw'] = df['Prob Draw'].apply(lambda x: 0 if x < 0 else x)

# Recalculating the Offensive and Defensive Indexes including shots on target and corners

# Offensive Index - combining goals, expected goals, shots on target, and corners
df['Home Offensive Index'] = (df['GF (Home)'] + df['xG_x (Home)'] + df['Home Shots on Target'] + df['Home Corners']) / 4
df['Away Offensive Index'] = (df['GF (Away)'] + df['xG_x (Away)'] + df['Away Shots on Target'] + df['Away Corners']) / 4

# Defensive Index - combining goals against, expected goals against, shots on target against, and corners against
# Lower values are better for defensive index, so this might need to be considered in the final performance index
df['Home Defensive Index'] = (df['GA (Home)'] + df['xGA (Home)'] + df['Away Shots on Target'] + df['Away Corners']) / 4
df['Away Defensive Index'] = (df['GA (Away)'] + df['xGA (Away)'] + df['Home Shots on Target'] + df['Home Corners']) / 4

# Overall Team Performance Index - a higher offensive index is good, but a higher defensive index is bad, so it's subtracted
df['Home Performance Index'] = df['Home Offensive Index'] - df['Home Defensive Index']
df['Away Performance Index'] = df['Away Offensive Index'] - df['Away Defensive Index']


# Creating interaction features between offensive and defensive indices
df['Home Off-Def Interaction'] = df['Home Offensive Index'] * df['Home Defensive Index']
df['Away Off-Def Interaction'] = df['Away Offensive Index'] * df['Away Defensive Index']

# Transforming probabilities into a feature capturing expected dominance
df['Expected Dominance'] = df['Prob Home Win'] - df['Prob Away Win']

# Calculating performance differential
df['Performance Differential'] = df['Home Performance Index'] - df['Away Performance Index']

# Calculating the Home Dominance Score
df['Home Dominance Score'] = df['Prob Home Win'] - df['Prob Away Win']


# Calculating the Recent Form Index for home and away teams
df['Home Recent Form Index'] = (df['Home Win %'] * 1) + (df['Home Draw %'] * 0.5) - (df['Home Loss %'] * 1)
df['Away Recent Form Index'] = (df['Away Win %'] * 1) + (df['Away Draw %'] * 0.5) - (df['Away Loss %'] * 1)


final_renamed_file_path = "final_all_euro_data_with_rankings_features.csv"
df.to_csv(final_renamed_file_path, index=False)
